## Reconcile API Usage with CSV Usage

Load the usage as reported by the API and reconcile with the usage as given on the billing CSV.

This notebook takes the results from two other notebooks and reconciles the reported usage 
differences. One of the source notebooks, [Load Azure Usage CSV.ipynb](Load%20Azure%20Usage%20CSV.ipynb), uses the billing CSV export feature of the Azure admin portal. 
The other, [Load Azure Daily Usage for Month to Match Invoice.ipynb](Load%20Azure%20Daily%20Usage%20for%20Month%20to%20Match%20Invoice.ipynb),  uses an Azure API. Currently, this reconcilliation reports some differences for the two 
sources that are likely due to discounts or included usage credits for the affected resources. 

In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
# load data processed from related notebooks

# data from CSV downloaded from Azure portal
df_daily_usage_csv = pickle.load( open( "df_daily_usage.p", "rb" ) )

# data from Azure billing API Azure
df_invoice_daily_usage = pickle.load( open( "df_invoice_daily_usage.p", "rb" ) )

In [3]:
df_daily_usage_csv.columns

Index(['Usage Date', 'Meter Category', 'Meter Id', 'Meter Sub-category',
       'Meter Name', 'Meter Region', 'Unit', 'Consumed Quantity',
       'Resource Location', 'Consumed Service', 'Resource Group',
       'Instance Id', 'Tags', 'Additional Info', 'Service Info 1',
       'Service Info 2'],
      dtype='object')

In [4]:
df_invoice_daily_usage.columns

Index(['infoFields', 'instanceData', 'meterCategory', 'meterId', 'meterName',
       'meterRegion', 'meterSubCategory', 'quantity', 'subscriptionId', 'unit',
       'usageEndTime', 'usageStartTime'],
      dtype='object')

In [5]:
# CSV has spurious quotes 
# todo: move this processing to source notebook
df_daily_usage_csv['Meter Id'] = df_daily_usage_csv['Meter Id'].str.replace('"', '')

In [6]:
# compare the two datasets joining on resource and usage date
result = pd.merge(df_daily_usage_csv,
    df_invoice_daily_usage,
    left_on=['Usage Date','Meter Id'], right_on = ['usageStartTime','meterId'],
    how='outer', 
    indicator=True)

In [7]:
result.columns

Index(['Usage Date', 'Meter Category', 'Meter Id', 'Meter Sub-category',
       'Meter Name', 'Meter Region', 'Unit', 'Consumed Quantity',
       'Resource Location', 'Consumed Service', 'Resource Group',
       'Instance Id', 'Tags', 'Additional Info', 'Service Info 1',
       'Service Info 2', 'infoFields', 'instanceData', 'meterCategory',
       'meterId', 'meterName', 'meterRegion', 'meterSubCategory', 'quantity',
       'subscriptionId', 'unit', 'usageEndTime', 'usageStartTime', '_merge'],
      dtype='object')

In [8]:
# add a percent differnce column to compare the usage 
result['pct_diff'] = result.apply(lambda row: (row['Consumed Quantity'] - row['quantity']) / row['quantity'], axis=1)

In [9]:
# The resouces and dates match if all rows '_merge' is both. _merge would 
# be 'right only' or 'left only' if a given resouce-date tuple is only in the CSV or
 # only in the API
result[['Usage Date', 'Meter Id', 'usageStartTime', 'meterId', 'Consumed Quantity', 'quantity', 'pct_diff', '_merge']]

,Usage Date,Meter Id,usageStartTime,meterId,Consumed Quantity,quantity,pct_diff,_merge
0,2018-06-12,e9549cbe-02d9-4213-b4be-22d6dfe8a3af,2018-06-12,e9549cbe-02d9-4213-b4be-22d6dfe8a3af,0.001389,0.001389,0.000000,both
1,2018-06-12,9cb0bde8-bc0d-468c-8423-a25fe06779d3,2018-06-12,9cb0bde8-bc0d-468c-8423-a25fe06779d3,0.000500,0.000500,0.000000,both
2,2018-06-12,9995d93a-7d35-4d3f-9c69-7a7fea447ef4,2018-06-12,9995d93a-7d35-4d3f-9c69-7a7fea447ef4,0.059841,0.059841,0.000000,both
3,2018-06-12,d101de3e-ae70-48bb-8605-64fcd0a3ce8f,2018-06-12,d101de3e-ae70-48bb-8605-64fcd0a3ce8f,0.766682,0.766682,0.000000,both
4,2018-06-12,b9e5e77c-a0b3-4a2c-9b8b-57fa54f31c52,2018-06-12,b9e5e77c-a0b3-4a2c-9b8b-57fa54f31c52,0.000100,0.000100,0.000000,both
5,2018-06-12,d54686f0-77ff-43f3-9e7c-2099030d32a7,2018-06-12,d54686f0-77ff-43f3-9e7c-2099030d32a7,0.000804,0.000804,0.000000,both
6,2018-06-12,f114cb19-ea64-40b5-bcd7-aee474b62853,2018-06-12,f114cb19-ea64-40b5-bcd7-aee474b62853,0.900000,0.900000,0.000000,both
7,2018-06-12,c80a3636-2edb-4248-bcb1-04ef818a75ac,2018-06-12,c80a3636-2edb-4248-bcb1-04ef818a75ac,0.009300,0.009300,0.000000,both
8,2018-06-12,923978e1-fd3f-4bd5-a798-f4b533057e46,2018-06-12,923978e1-fd3f-4bd5-a798-f4b533057e46,0.004500,0.004500,0.000000,both
9,2018-06-12,32c3ebec-1646-49e3-8127-2cafbd3a04d8,2018-06-12,32c3ebec-1646-49e3-8127-2cafbd3a04d8,1.619603,1.619603,0.000000,both


In [10]:
# show results with percent difference between CSV source and API source > 0.0
result.loc[result['pct_diff'] > 0.000001][['Usage Date', 'Meter Id', 'usageStartTime', 'meterId', 'Consumed Quantity', 'quantity', 'pct_diff', '_merge']]

,Usage Date,Meter Id,usageStartTime,meterId,Consumed Quantity,quantity,pct_diff,_merge
36,2018-06-16,65d4ded2-41ae-43a8-bb68-3c200e1ba864,2018-06-16,65d4ded2-41ae-43a8-bb68-3c200e1ba864,36.000000,20.000000,0.800000,both
38,2018-06-16,65d4ded2-41ae-43a8-bb68-3c200e1ba864,2018-06-16,65d4ded2-41ae-43a8-bb68-3c200e1ba864,36.000000,20.000000,0.800000,both
46,2018-06-16,38529ded-5b8e-4b49-b078-ce81794a3543,2018-06-16,38529ded-5b8e-4b49-b078-ce81794a3543,20.000000,16.000000,0.250000,both
49,2018-06-16,38529ded-5b8e-4b49-b078-ce81794a3543,2018-06-16,38529ded-5b8e-4b49-b078-ce81794a3543,36.000000,16.000000,1.250000,both
50,2018-06-16,38529ded-5b8e-4b49-b078-ce81794a3543,2018-06-16,38529ded-5b8e-4b49-b078-ce81794a3543,36.000000,20.000000,0.800000,both
58,2018-06-17,505db374-df8a-44df-9d8c-13c14b61dee1,2018-06-17,505db374-df8a-44df-9d8c-13c14b61dee1,1.283423,0.383067,2.350388,both
67,2018-06-17,65d4ded2-41ae-43a8-bb68-3c200e1ba864,2018-06-17,65d4ded2-41ae-43a8-bb68-3c200e1ba864,240.000000,96.000000,1.500000,both
68,2018-06-17,65d4ded2-41ae-43a8-bb68-3c200e1ba864,2018-06-17,65d4ded2-41ae-43a8-bb68-3c200e1ba864,240.000000,96.000000,1.500000,both
81,2018-06-17,38529ded-5b8e-4b49-b078-ce81794a3543,2018-06-17,38529ded-5b8e-4b49-b078-ce81794a3543,240.000000,96.000000,1.500000,both
82,2018-06-17,38529ded-5b8e-4b49-b078-ce81794a3543,2018-06-17,38529ded-5b8e-4b49-b078-ce81794a3543,240.000000,96.000000,1.500000,both
